In [1]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)


from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from nltk.corpus import stopwords
import pandas as pd
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.text import Tokenizer 

stops = set(stopwords.words('english'))

def load_dataset():
    df = pd.read_csv('D:/PESU/NLP/final/dataset/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Loading data...')
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

Loading data...


In [2]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))


# ENCODE REVIEW
token = Tokenizer(lower=False) 
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()
print(max_length)

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1
print(total_words)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

131
92511
40000 train sequences
10000 test sequences
Pad sequences (samples x time)
x_train shape: (40000, 131)
x_test shape: (10000, 131)


In [3]:
print('Build model...')

# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(128, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(128, return_sequences=True)) 
model.add(LSTM(LSTM_OUT))  # return a single vector of dimension 32
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())


Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 131, 32)           2960352   
                                                                 
 lstm (LSTM)                 (None, 131, 64)           24832     
                                                                 
 lstm_1 (LSTM)               (None, 131, 128)          98816     
                                                                 
 lstm_2 (LSTM)               (None, 131, 128)          131584    
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 3,265,057
Trainable params:

In [4]:
model.fit(x_train, y_train, batch_size = 128, epochs = 15, validation_data=(x_test, y_test))

Epoch 1/15
313/313 [==============================] - 64s 171ms/step - loss: 0.4450 - accuracy: 0.7707 - val_loss: 0.2974 - val_accuracy: 0.8764
Epoch 2/15
313/313 [==============================] - 52s 166ms/step - loss: 0.2030 - accuracy: 0.9275 - val_loss: 0.3047 - val_accuracy: 0.8811
Epoch 3/15
313/313 [==============================] - 52s 167ms/step - loss: 0.1189 - accuracy: 0.9618 - val_loss: 0.3613 - val_accuracy: 0.8775
Epoch 4/15
313/313 [==============================] - 52s 167ms/step - loss: 0.0783 - accuracy: 0.9767 - val_loss: 0.4193 - val_accuracy: 0.8734
Epoch 5/15
313/313 [==============================] - 52s 167ms/step - loss: 0.0505 - accuracy: 0.9858 - val_loss: 0.4395 - val_accuracy: 0.8678
Epoch 6/15
313/313 [==============================] - 52s 167ms/step - loss: 0.0512 - accuracy: 0.9859 - val_loss: 0.4928 - val_accuracy: 0.8629
Epoch 7/15
313/313 [==============================] - 52s 165ms/step - loss: 0.0460 - accuracy: 0.9872 - val_loss: 0.4221 - val_ac

In [5]:
score, acc = model.evaluate(x_test, y_test, batch_size=128)
print('Test score:', score)
print('Test accuracy:', acc)

79/79 [==============================] - 5s 61ms/step - loss: 0.6359 - accuracy: 0.8477
Test score: 0.6359463334083557
Test accuracy: 0.8476999998092651


In [6]:
model.save('vanillaLSTM')

INFO:tensorflow:Assets written to: vanillaLSTM\assets


INFO:tensorflow:Assets written to: vanillaLSTM\assets
